In [36]:
import cv2 as cv
from PIL import Image
import os
from tqdm import tqdm
import numpy as np
import random

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dropout,Input,Dense,Flatten
import tensorflow.keras as k
from tensorflow.keras.optimizers import Adam

import pickle

#### Parameters

In [11]:
dataset = []
labels = []
valid_type = ['png','jpg']
height = 70
width = 70

#### Creating the dataset

In [13]:
mouth_open_dir = os.listdir('./Dataset/Yawn/')

for img_id, each_image in enumerate(tqdm(mouth_open_dir)):
    if each_image.split('.')[1] in valid_type :
        image = cv.imread('./Dataset/Yawn/'+each_image)
        image = Image.fromarray(image,"RGB")
        image = image.resize((height, width))
        image = np.array(image)

        dataset.append(image)
        labels.append(1)

100%|██████████| 3254/3254 [00:10<00:00, 299.51it/s] 


In [14]:
mouth_close_dir = os.listdir('./Dataset/no_yawn/')

for img_id, each_image in enumerate(tqdm(mouth_close_dir)):
    if each_image.split('.')[1] in valid_type :
        image = cv.imread('./Dataset/no_yawn/'+each_image)
        image = Image.fromarray(image,"RGB")
        image = image.resize((height, width))
        image = np.array(image)

        dataset.append(image)
        labels.append(0)

100%|██████████| 3316/3316 [00:33<00:00, 98.45it/s] 


In [16]:
random_sequence = [i for i in range(len(labels))]
random.shuffle(random_sequence)

labels = [labels[i] for i in random_sequence]
dataset = [dataset[i] for i in random_sequence]

print(labels[:20])

[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1]


#### Dataset Split

In [19]:
x_train, x_test, y_train,y_test = train_test_split(dataset, to_categorical(np.array(labels)), test_size=0.2, random_state=42)
x_test, x_val, y_test,y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [22]:
print(len(x_train), y_train.shape)
print(len(x_test), y_test.shape)
print(len(x_val), y_val.shape)

5256 (5256, 2)
657 (657, 2)
657 (657, 2)


#### Model Creation

In [27]:
input_shape = (height, width, 3)

# input layer
inp = Input(shape=input_shape)

conv1 = Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(inp)
pool1 = MaxPool2D(pool_size=(2,2))(conv1)
norm1 = BatchNormalization(axis = -1)(pool1)
drop1 = Dropout(rate=0.2)(norm1)

conv2 = Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(drop1)
pool2 = MaxPool2D(pool_size=(2,2))(conv2)
norm2 = BatchNormalization(axis = -1)(pool2)
drop2 = Dropout(rate=0.2)(norm2)

conv3 = Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(drop2)
pool3 = MaxPool2D(pool_size=(2,2))(conv3)
norm3 = BatchNormalization(axis = -1)(pool3)
drop3 = Dropout(rate=0.2)(norm3)

# hidden layer
flat = Flatten()(drop3)

hidden1 = Dense(512, activation="relu")(flat)
norm4 = BatchNormalization(axis = -1)(hidden1)
drop4 = Dropout(rate=0.2)(norm4)

hidden2 = Dense(512, activation="relu")(drop4)
norm5 = BatchNormalization(axis = -1)(hidden2)
drop5 = Dropout(rate=0.2)(norm5)

# output layer
out = Dense(2, activation = 'sigmoid')(drop5)

In [31]:
model = k.Model(inputs = inp, outputs = out)

model.compile(optimizer = Adam(learning_rate = 0.001), loss="categorical_crossentropy", metrics=['accuracy'])

print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 70, 70, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 70, 70, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 35, 35, 32)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 35, 35, 32)        128       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 35, 35, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 35, 35, 32)        9248

 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_3 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 batch_normalization_4 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
          

#### Model Performance

In [34]:
model.fit(np.array(x_train), y_train, epochs = 10, batch_size = 64, verbose=1, validation_data=(np.array(x_val), y_val))

Epoch 1/10


83/83 [==============================] - 14s 110ms/step - loss: 0.4362 - accuracy: 0.8291 - val_loss: 0.2944 - val_accuracy: 0.8554
Epoch 2/10
83/83 [==============================] - 11s 129ms/step - loss: 0.2909 - accuracy: 0.8788 - val_loss: 0.3109 - val_accuracy: 0.8904
Epoch 3/10
83/83 [==============================] - 8s 98ms/step - loss: 0.2264 - accuracy: 0.9047 - val_loss: 0.2912 - val_accuracy: 0.8995
Epoch 4/10
83/83 [==============================] - 8s 95ms/step - loss: 0.2053 - accuracy: 0.9150 - val_loss: 0.3548 - val_accuracy: 0.8965
Epoch 5/10
83/83 [==============================] - 8s 95ms/step - loss: 0.1772 - accuracy: 0.9268 - val_loss: 0.2999 - val_accuracy: 0.9041
Epoch 6/10
83/83 [==============================] - 8s 95ms/step - loss: 0.1438 - accuracy: 0.9404 - val_loss: 0.3026 - val_accuracy: 0.8995
Epoch 7/10
83/83 [==============================] - 8s 95ms/step - loss: 0.1247 - accuracy: 0.9513 - val_loss: 0.1916 - val_accuracy: 0.9437
Epoch 8

In [35]:
print("Accuracy",model.evaluate(np.array(x_test), y_test)[1]*100)

21/21 [==============================] - 1s 22ms/step - loss: 0.2753 - accuracy: 0.9254
Accuracy 92.54185557365417


#### Model Save

In [37]:
with open('mouth_state_classification_model.pkl','wb') as f:
    pickle.dump(model, f)